# Activities

This document guides you through the steps necessary to create your first model and to connect it to a simple application. As discussed in the theory lesson models are fundamental to have a structured image of your data in the database. Later the model will be re-used in a more sophisticated solution where users will be able to access and interact with it via browser. We will get there soon, but first let's start from the basics.

 
# Step 0: .NET Core
- Install the .NET Core SDK from https://www.microsoft.com/net/core.
- As IDE we recommend Visual Studio Code https://code.visualstudio.com/download. Remember, if you decide to install this IDE, then use the extentions manager tool to install the C# interpreter, which will enhance the overall experience of your IDE when programming in C#. C# will be used throughout all this course.
    
# Step 1: Project setup
- Move to your projects folder and run the following instructions to set up the application.

``` bash
dotnet new console -o [ProjectName] (think of suitable name)
cd [ProjectName]
dotnet restore
```
- To use the Entity Framework (EF) we need to install it first in our project. The EF framework provides us with the DbContext and migration scripts that are necessary to interact and maintain our models (NOTE, you need PostgreSQL and pgAdmin installed on our machine).

``` bash
dotnet add package Microsoft.EntityFrameworkCore.Design
dotnet add package Npgsql.EntityFrameworkCore.PostgreSQL
```

- Open now the application with your IDE and add to the file $\textbf{[ProjectName].csproj}$ the following item to the project.

``` xml
<ItemGroup>
  <DotNetCliToolReference Include="Microsoft.EntityFrameworkCore.Tools.DotNet" Version="2.0.0" />
</ItemGroup>
```
    - Eventually your C# project (csproj) should look like the following:
``` xml
<Project Sdk="Microsoft.NET.Sdk">
  <PropertyGroup>
    <OutputType>Exe</OutputType>
    <TargetFramework>netcoreapp2.0</TargetFramework>
  </PropertyGroup>
  <ItemGroup>
    <PackageReference Include="Microsoft.EntityFrameworkCore.Design" Version="2.0.0" />
    <PackageReference Include="Npgsql.EntityFrameworkCore.PostgreSQL" Version="2.0.0" />
  </ItemGroup>
  <ItemGroup>
    <DotNetCliToolReference Include="Microsoft.EntityFrameworkCore.Tools.DotNet" Version="2.0.0" />
  </ItemGroup>
</Project>
```

- Run again `dotnet restore`


- To check if the installation is so far good execute the following command on your terminal: `dotnet ef -h (or --help)
`. You should be able to see a series of options on your terminal regarding the EF.



# Step 2: Entity framework Context and schema creation one<>many

- The Entity Framework enables you to query, insert, update, and delete data, by manipulating common language runtime (CLR) objects known as entities (or POCOs). Given a model (which is written in C#) the EF maps its entities and relationships into an actual database. In this example we chose as database Postgres. The EF provides facilities to interact with the records in the database as if they are .Net objects (in our case C# objects). Moreover, EF solves typical challenges found in the DB realm such as concurrency, persistence of the data (between the .Net objects and the database records), etc.


- The primary class that is responsible for interacting with the database is $System.Data.Entity.DbContext$ (often referred as context).


- In the following we chose to model a simple movie application, where a movie is played by many actors.


- Add to the root of the project a C# file called $Model.cs$ with the following content (the comments are only meant as explanation).


``` csharp
using Microsoft.EntityFrameworkCore;
using System.Collections.Generic;

namespace Model
{
  public class MovieContext : DbContext
  {
    //this is actual entity object linked to the movies in our DB
    public DbSet<Movie> Movies { get; set; }
    //this is actual entity object linked to the actors in our DB
    public DbSet<Actor> Actors { get; set; }

    //this method is run automatically by EF the first time we run the application
    protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder){
      //here we define the name of our database
      optionsBuilder.UseNpgsql("User ID=postgres;Password=;Host=localhost;Port=5432;Database=MovieDB;Pooling=true;");
    }
  }

  //this is the typed representation of a movie in our project
  public class Movie
  {
    public int Id { get; set; }
    public string Title { get; set; }
    public List<Actor> Actors { get; set; }
  }
 
  //this is the typed representation of an actor in our project
  public class Actor
  {
    public int Id { get; set; }
    public string Name { get; set; }
    public int MovieId { get; set; }
  }
}
```
 
- This file represents the model of your application. See that a movie contains a series of actors. As with DB each entity above has an attribute $Id$ which later will be used as a primary key. Moreover, EF recognizes automatically the keys in the actors list in movie as foreign keys.


- Our first model is now ready. Now run the following commands to create a migration, the initial set of tables for the model, and to apply the changes to the database.

``` bash
dotnet ef migrations add InitialCreateMovieDb
dotnet ef database update
```

- Remember to save files after modifying them (before executing any command in the terminal).

# Step 3: Inserting data into the database

- Now that our DB is ready we can interact with it in our project by simply instantiating an object of our $MovieContext$ class. We start by inserting our first movie. In order to insert a new movie into the database you need to instantiate first the movie and its actors and then add it to the database. Remember that to make your changes persistent in the database, you need to call the method SaveChanges() in context.


- Now open the Program.cs file and add the following code to the main-method (remember to add $using Model;$ on the top of this file).


``` csharp
...
using System.Linq;
...
using (var db = new MovieContext())
{
  
  Movie m = new Movie
  {
    Title = "No country for old men",
    Actors = new System.Collections.Generic.List<Actor> {
      new Actor{Name = "Tommy Lee"},
      new Actor{Name = "Xavier Berdem"}
    }
  };
  db.Movies.Add(m);
  db.SaveChanges();
  }
}
```


- Note the usage of `using`. Here `using` will take care of closing the connection with the database once its body is executed.


- If you run `dotnet run` the code above will be executed and the movie above (together with its actors) will be inserted into the database.


- If you want to see the actors for each movie you can also run in your main method the following code. Note the foreign key MovieId of actor is used as filter to select the actors of a specific movie. 


``` csharp
...
using (var db = new MovieContext())
{
  foreach (var movie in db.Movies)
  {
    Console.WriteLine("Found movie with title" + movie.Title);
    foreach (var actor in db.Actors.Where(a => movie.Id == a.MovieId))
    {
      Console.WriteLine("Found actor with name" + actor.Name);
    }
  }
}
```


# Step 4: Modifying inserted data 

- In order to update existing data in a table the EF core provides you with a find-method that takes as input an id (our key) of the object to find.


- In the following we modify the movie inserted above by changing its title.


``` csharp
using (var db = new MovieContext())
{
  Movie foundMovie = db.Movies.Find(1);
  Console.WriteLine("Found movie with title" + foundMovie.Title);
  foundMovie.Title = "White cats, Black cats...";
  db.SaveChanges();
  Console.WriteLine("Title changed");
}
```

- If you run `dotnet run` the code above will be executed and the title of the movie changes.



# Step 5: Entity framework Context and schema creation many<>many 

- We have to consider, however, that an actor can play in different movies. A many-to-many relationship is represented by including an entity class for the join table and mapping two separate one-to-many relationships in your Model.cs file. 

``` csharp
...
//this is the typed representation of our join table mapping a movie with an actor
public class MovieActor{
  public int MovieId { get; set; }
  public Movie Movie { get; set; }
  public int ActorId { get; set; }
  public Actor Actor { get; set; }
}
```

- Also the other two classes Movie and Actor need a little modification to accomodate this change.


``` csharp
public class Movie
 {
  public int Id { get; set; }
  public string Title { get; set; }
  public virtual List<MovieActor> Actors { get; set; }
}

public class Actor
{
  public int Id { get; set; }
  public string Name { get; set; }
  public virtual List<MovieActor> Movies { get; set; }
}
```

- The mapping rules are implemented by altering the DbContext class and adding a method called OnModelCreating(ModelBuilder modelBuilder) that tells EF that a movie has many actors and vice-versa.

``` csharp
public class MovieContext : DbContext
{
  
  DbSet<MovieActor> MovieActor { get; set; }
  ...
  protected override void OnModelCreating(ModelBuilder modelBuilder)
  {
    modelBuilder.Entity<MovieActor>()
      .HasKey(t => new { t.ActorId, t.MovieId });
    modelBuilder.Entity<MovieActor>()
      .HasOne(ma => ma.Movie)
      .WithMany(m => m.Actors)
      .HasForeignKey(ma => ma.MovieId);
    modelBuilder.Entity<MovieActor>()
      .HasOne(ma => ma.Actor)
      .WithMany(m => m.Movies)
      .HasForeignKey(ma => ma.ActorId);
  }
}
...
```
    

- In the command line execute the following commands to test the new changes. In case you get an error while updating, then remove the migration folder and the database, and run again all the below instructions except for the first one (the drop).


``` bash
dotnet ef database drop
dotnet ef migrations add "N-N"
dotnet ef database update
dotnet run 
```

- After running those commands, the database will be empty (because of drop). To add again (and print) all movies and actors following the new schema you need of course to adapt your main (and run the project again) so that it includes the new changes.

- As an additional exercise add at least 5 movies, each with at least 3 actors (remember an actor now can play in different movies) and print all the movies and their respective actors.



    
- Additional resources can be found here: 
    - Source1: https://docs.microsoft.com/en-us/ef/core/get-started/netcore/new-db-sqlite
    - Source2: https://docs.microsoft.com/en-us/ef/core/modeling/relationships




